In [1]:
%serialconnect

No possible ports foundConnecting to --port=/dev/ttyUSB0 --baud=115200 could not open port /dev/ttyUSB0: [Errno 2] No such file or directory: '/dev/ttyUSB0'

Are you sure your ESP-device is plugged in?

In [2]:
%sendtofile --quiet --source ../basicsockets/syslog.py
%sendtofile --quiet --source ../essential_sensor_code/asyncdevices.py

%sendtofile --quiet --source ../wificodes.txt
%sendtofile --quiet --source ../basicsockets/connectwifi.py
%sendtofile --quiet --source ../basicsockets/connecthotspot.py

%sendtofile --quiet --source ../async_web_serve/webserve_funcs.py


Sent 34 lines (705 bytes) to syslog.py.
Sent 48 lines (1507 bytes) to asyncdevices.py.
Sent 34 lines (1042 bytes) to connectwifi.py.
Sent 9 lines (327 bytes) to connecthotspot.py.
Sent 190 lines (6050 bytes) to webserve_funcs.py.


In [46]:
%sendtofile --source uploadfiletest.html


Sent 13 lines (338 bytes) to uploadfiletest.html.


In [10]:
%comment This does the main file loading

%sendtofile --source plain_main.py main.py

# you need to reset properly and monitor on serial line through picocom, sa

Sent 53 lines (1478 bytes) to main.py.


In [ ]:
# Below are the lines you can use to build the webserver in immediate mode

In [11]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.9.2-443-g236297f4 on 2017-11-14; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft reboot\r\nTraceback (most recent call last):\r\n  File "main.py", line 2, in <module>\r\nKeyboardInterrupt: \r\nMicroPython v1.9.2-443-g236297f4 on 2017-11-14; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [47]:
%disconnect


Closing serial Serial<id=0x7fae9b3f5518, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [3]:
#%%writefile plain_main.py

import time
time.sleep(1.0)

from syslog import log, elog, powid
import os, ustruct, uasyncio
uasyncio.set_debug(0)

#from connectwifi import ipnumber
from connecthotspot import ipnumber

from webserve_funcs import servestaticfile
from webserve_funcs import servefilelist
from webserve_funcs import readhttpheaders
from webserve_funcs import recpostsave
from webserve_funcs import servemessage
from webserve_funcs import convertsavepostfile



LOG 834 ON 0
Creating access point ESP_0efe91
Device has ipnumber 192.168.4.1


In [4]:
#%%writefile --append plain_main.py

async def handleconnection(reader, writer):
    receivedrequest = await readhttpheaders(reader)
    
    print(receivedrequest)
    if "POST" in receivedrequest:
        await recpostsave(receivedrequest, reader, "POSTUPLOAD.txt")
        await servemessage(writer, "done")
        await writer.aclose()
        convertsavepostfile("POSTUPLOAD.txt")
        return

    rpath = receivedrequest.get("GET")
    log(str(receivedrequest))
    
    bcallaclose = True
    if rpath == "":
        await servefilelist(writer)
    elif rpath:
        await servestaticfile(writer, rpath)
        
    if bcallaclose:
        await writer.aclose()



In [5]:
#%%writefile --append plain_main.py


# enables rewriting handleconnection function without restarting
def handleconnection_indirect(reader, writer):
    try:
        return handleconnection(reader, writer)
    except OSError as e:
        print("handleconnection exception", str(e))

port = 80
loop = uasyncio.get_event_loop()
print("* Running on http://%s:%s/" % (ipnumber, port))
loop.create_task(uasyncio.start_server(handleconnection_indirect, ipnumber, port))


* Running on http://192.168.4.1:80/


In [6]:
#%%writefile --append plain_main.py

loop.run_forever()


.{'GET': ''}
LOG 888 {'GET': ''} 
{'GET': 'favicon.ico'}
LOG 888 {'GET': 'favicon.ico'} 
LOG 888 File 'favicon.ico' not found 
{'GET': 'syslog.py'}
LOG 892 {'GET': 'syslog.py'} 
LOG 892 File 'syslog.py' (705 bytes) served in 11ms 
{'GET': 'favicon.ico'}
LOG 892 {'GET': 'favicon.ico'} 
LOG 892 File 'favicon.ico' not found 
{'GET': 'favicon.ico'}
LOG 894 {'GET': 'favicon.ico'} 
LOG 894 File 'favicon.ico' not found 
.......................................................

*** Sending Ctrl-C



Keyboard interrupt while waiting response on Ctrl-C



In [244]:
%sendtofile --source plain_main.py main.py

Sent 56 lines (1533 bytes) to main.py.


In [35]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [23]:
%sendtofile --source ../htmlfiles/uploadfiletest.html

Sent 13 lines (338 bytes) to uploadfiletest.html.


In [26]:
%sendtofile --source lighthttpserve_funcs.py

Sent 182 lines (5575 bytes) to lighthttpserve_funcs.py.


In [36]:
from machine import Pin
Pin(12, Pin.OUT).value(0)

import network

si = network.WLAN(network.AP_IF) # create access-point interface
macaddress = "".join("{:02x}".format(x)  for x in si.config("mac"))
si.active(True)         # activate the interface
print("Creating access point ESP_{}".format(macaddress[-6:]))

ipnumber = si.ifconfig()[0]
print("Device has ipnumber", ipnumber)


[leftinbuffer] ['ip:192.168.43.42,mask:255.255.255.0,gw:192.168.43.1']
Creating access point ESP_4aff86
Device has ipnumber 192.168.4.1


In [37]:
import uasyncio as asyncio
from lighthttpserve_funcs import servestaticfile, servefilelist, readhttpheaders, recpostsave, servemessage, convertsavepostfile

async def handleconnection(reader, writer):
    receivedrequest = await readhttpheaders(reader)
    
    if "POST" in receivedrequest:
        await recpostsave(receivedrequest, reader, "POSTUPLOAD.txt")
        await servemessage(writer, "done")
        await writer.aclose()
        convertsavepostfile("POSTUPLOAD.txt")
        return

    rpath = receivedrequest.get("GET")
    log(receivedrequest)
    
    bcallaclose = True
    if rpath == "":
        await servefilelist(writer)
    elif rpath:
        await servestaticfile(writer, rpath)
        
    if bcallaclose:
        await writer.aclose()

        
# enables rewriting handleconnection function without restarting
def handleconnection_indirect(reader, writer):
    try:
        return handleconnection(reader, writer)
    except OSError as e:
        print("handleconnection exception", str(e))

port = 80
loop = asyncio.get_event_loop()
print("* Running on http://%s:%s/" % (ipnumber, port))
loop.create_task(asyncio.start_server(handleconnection_indirect, ipnumber, port))


* Running on http://192.168.4.1:80/


In [38]:
log = print
loop.run_forever()

.{'GET': ''}
..{'GET': 'uploadfiletest.html'}
File 'uploadfiletest.html' (338 bytes) served in 6ms
....recpostsave 6478
convpostfile 27pic2.jpg
.where the prelinebreak?
{'GET': ''}
.{'GET': '27pic2.jpg'}
File '27pic2.jpg' (6143 bytes) served in 123ms
.{'GET': 'POSTUPLOAD.txt'}
File 'POSTUPLOAD.txt' (6478 bytes) served in 132ms
.....{'GET': 'uploadfiletest.html'}
File 'uploadfiletest.html' (338 bytes) served in 6ms
.......recpostsave 1344
convpostfile installer.py
.{'GET': ''}
.{'GET': 'POSTUPLOAD.txt'}
File 'POSTUPLOAD.txt' (1344 bytes) served in 35ms
......
**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)


**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)



In [16]:
%disconnect

serial exception on close write failed: [Errno 5] Input/output error

Closing serial Serial<id=0x7f7f5ecf0780, open=True>(port='/dev/ttyUSB1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
